In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, HeatMap

import matplotlib.pyplot as plt
plt.style.use('bmh')

import seaborn as sns
import osmnx as ox
import fiona

import os
os.environ["PROJ_LIB"] = "C:\ProgramData\Anaconda3\Library\share" #window

In [ ]:
#Filepaths
work = r"D:"
laptop = r"C:\Users\jerem"
origin = work

In [ ]:
fp =  r"\Box Sync\Policy Evaluation\working_data\analytical_panel_shapefile\analytical_panel_qt_shapefile.shp"
df = gpd.read_file(origin+fp)

Current CRS is set for New York Long Island (ftUS)

In [ ]:
df.crs

Switch to WGS 84

In [ ]:
df = df.to_crs({'init': 'epsg:4326'})

Collapse into counts of collisions, number of injuries by intersections

In [ ]:
columns = df.columns

In [ ]:
print(columns)

In [ ]:
new_cols = ['collision_count','personsinjured','pedestriansinjured','cyclistinjured','motoristinjured','bronx','brooklyn','manhattan','queens','statenisland','distance_to_LPIS','distance_to_Street','distance_to_Sch','distance_to_LTC','distance_to_bikeroute','distance_to_truckroute','flag_left_turn_ever','flag_left_turn','flag_LPIS_ever','flag_LPIS','bike_route_tv','bike_route_ever', 'flag_seniors','flag_priorityinters','flag_school','quarter','year','intersection_id','y','x','geometry']
df.set_axis(new_cols, axis='columns', inplace=False)



In [ ]:
df.columns

In [ ]:
df['longitude'] = df.geometry.x
df['latitude'] = df.geometry.y

In [ ]:
df.head(2)

In [ ]:
lastcols = ['intersection_id', 'flag_LPIS_ever', 'latitude','longitude'] 
sumcols = ['collision_count', 'personsinjured', 'pedestriansinjured', 'cyclistinjured', 'motoristinjured'] 
allcols = lastcols + sumcols
methods = len(lastcols)*['last'] + len(allcols)*['sum']
aggdict = dict(zip(allcols, methods))
aggdict

In [ ]:
df1 = df.groupby('intersection_id')[allcols].agg(aggdict)
df1.shape

In [ ]:
df1.head(2)

In [ ]:
(df1.groupby('flag_LPIS_ever')[['intersection_id','personsinjured', 'pedestriansinjured', 'cyclistinjured', 'motoristinjured']]
       .agg({'intersection_id': 'count', 'personsinjured' : 'mean', 'pedestriansinjured' : 'mean', 'cyclistinjured' : 'mean', 'motoristinjured' : 'mean'}) )

In [ ]:
plt.title('Total Persons injured over entire period')
df1.personsinjured.hist(bins=[1,2,3,4,5,10, 15, 20, 30, 50]);

In [ ]:
m = folium.Map(location=[40.736480, -73.927184], zoom_start =15, tiles='OpenStreetMap')

for idx, row in df1.iterrows():
    if (row.flag_LPIS_ever == 1):
        folium.Circle(location=[row.latitude, row.longitude],
                     popup=f'LPIS {row.intersection_id}',
                     radius = 10 + row.personsinjured*3
                     ).add_to(m)
    else:
       folium.Circle(location=[row.latitude, row.longitude],
                     popup=f'{row.intersection_id}',
                     radius = 10 + row.personsinjured*3,
                     color = 'grey',
                     ).add_to(m)
#m

In [ ]:
m.save('m.html')

In [ ]:
Heatmap

In [ ]:
df2 = folium.Map(location=[40.736480, -73.927184], zoom_start =15, tiles='OpenStreetMap')

HeatMap(list(zip(df1.latitude, df1.longitude, df1.personsinjured)), radius = 12).add_to(df2) 

for idx, row in df1.iterrows():
    if (row.flag_LPIS_ever == 1):
        folium.Circle(location=[row.latitude, row.longitude],
                     popup=f'LPIS {row.intersection_id}',
                     radius = 10 + row.personsinjured*3
                     ).add_to(df2)

#df2

In [ ]:
df2.save('df2.html')

A slippy map of all LPIS intersections
We use the Folium MarkerCluster option so that the markers 'cluster' when we zoom out rather than overwhelm the map.

Collapse the entire panel to a dataframe with just LPIS intersections. We can use the same columns and dictionary as used above.

In [ ]:
df3  = df.groupby('intersection_id')[allcols].agg(aggdict)

In [ ]:
LPIS = (df.flag_LPIS_ever == 1)

In [ ]:
df[LPIS].shape

In [ ]:
locations = list(zip(df[LPIS].latitude, df[LPIS].longitude,))
locations[:4]

In [ ]:
map2 = folium.Map(location=[40.736480, -73.927184], zoom_start =13, tiles='OpenStreetMap')
marker_cluster = MarkerCluster().add_to(map2)

In [ ]:
for idx, row in df[LPIS].iterrows():
    folium.Marker(location=[row.latitude, row.longitude],
                 popup=f'ID={row.intersection_id}, inj ={row.personsinjured}'
                 ).add_to(marker_cluster)

In [ ]:
map2.save('lpis.html')


In [ ]:
#map2